In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import ipywidgets as widgets
import numpy as np

In [2]:
df = pd.read_csv('results/oracle_revamp_eval.csv')


In [6]:
if False:
	df = pd.read_csv('results/oracle_revamp_eval.csv')
	df = df[df["oracle_class"] != "InternLMOracle"]
	df.to_csv('results/oracle_revamp_eval.csv')

In [3]:
summary = []
oracles = ['BinaryOracle', 'MutationOracle', 'Mutation1Oracle', 'SoloOracle', 'RelativeOracle', 'DiffOracle', 'ExampleOracle']
judges = {'Meta-Llama-3-8B-Instruct-q8_0': '8B', 'Meta-Llama-3.1-70B-Instruct-q8_0': '70B', 'Meta-Llama-3.1-70B-Instruct-WQE-0.1-q8_0': 'WQE'}
indices = []
for oracle in oracles:
	for judge in judges:
		df_oracle = df[(df['oracle_class'] == oracle) & (df['explain'] == False) & (df['judge_name'] == judge)]
		avg = df_oracle['pred_correct'].mean()
		time = df_oracle['time_taken'].mean()
		summary.append([avg, time, avg/time])
		print(oracle, judges[judge], len(df_oracle))
		indices.append(f"{oracle}_{judges[judge]}")

oracles2 = ['ArmoRMOracle', 'InternLMOracle']
for oracle in oracles2:
	df_oracle = df[(df['oracle_class'] == oracle) & (df['explain'] == False)]
	avg = df_oracle['pred_correct'].mean()
	time = df_oracle['time_taken'].mean()
	summary.append([avg, time, avg/time])
	print(oracle, judges[judge], len(df_oracle))
	indices.append(f"{oracle}")


oracle_scoring = pd.DataFrame(data=summary,index=indices, columns=['accuracy', 'time', 'value'])
oracle_scoring

BinaryOracle 8B 106
BinaryOracle 70B 106
BinaryOracle WQE 106
MutationOracle 8B 106
MutationOracle 70B 106
MutationOracle WQE 106
Mutation1Oracle 8B 106
Mutation1Oracle 70B 106
Mutation1Oracle WQE 106
SoloOracle 8B 106
SoloOracle 70B 106
SoloOracle WQE 106
RelativeOracle 8B 106
RelativeOracle 70B 106
RelativeOracle WQE 106
DiffOracle 8B 95
DiffOracle 70B 95
DiffOracle WQE 95
ExampleOracle 8B 106
ExampleOracle 70B 106
ExampleOracle WQE 106
ArmoRMOracle WQE 116
InternLMOracle WQE 116


,accuracy,time,value
BinaryOracle_8B,0.632075,0.193683,3.263447
BinaryOracle_70B,0.650943,1.212275,0.536960
BinaryOracle_WQE,0.632075,1.202802,0.525503
MutationOracle_8B,0.330189,0.407907,0.809470
MutationOracle_70B,0.518868,2.644821,0.196183
MutationOracle_WQE,0.377358,2.630466,0.143457
Mutation1Oracle_8B,0.452830,0.202115,2.240458
Mutation1Oracle_70B,0.698113,1.295204,0.538999
Mutation1Oracle_WQE,0.424528,1.292276,0.328512
SoloOracle_8B,0.415094,0.381081,1.089255


In [5]:
def indexer(value):
    mapping = {'ResponseQuality.A_BETTER': 0, 'ResponseQuality.B_BETTER': 1, 'ResponseQuality.TIE': 2}
    return mapping[value.strip()]

def create_table(df, label=None, pred=None, oracle=None, explain=None, judge=None):
	if oracle != None:
		df = df[(df['oracle_class'] == oracle)]
	if explain != None:
		df = df[(df['explain'] == explain)]
	if judge != None:
		df = df[(df['judge_name'] == judge)]

	if label == "A":
		df = df[(df['original_label'] == 'ResponseQuality.A_BETTER')]
	elif label == "B":
		df = df[(df['original_label'] == 'ResponseQuality.B_BETTER')]
	elif label == "TIE":
		df = df[(df['original_label'] == 'ResponseQuality.TIE')]

	if pred == 1:
		df = df[(df['pred_correct'] == 1)]
	elif pred == .5:
		df = df[(df['pred_correct'] == .5)]
	elif pred == 0:
		df = df[(df['pred_correct'] == 0)]

	init_data = [[0,0,0], [0,0,0], [0,0,0]]
	for index, row in df.iterrows():
		
		i = indexer(row["original_pred"])
		j = 0
		if row["oracle_class"] not in ["SoloOracle", "BinaryOracle", "Mutation1Oracle", "DiffOracle"]:
			j = indexer(row["followup_pred"])
		init_data[i][j] += 1
	return pd.DataFrame(data=init_data,index=['A', 'B', 'TIE'], columns=['A', 'B', 'TIE'])
	

create_table(df,oracle='DiffOracle', explain=False, judge="Meta-Llama-3.1-70B-Instruct-q8_0")

,A,B,TIE
A,52,0,0
B,43,0,0
TIE,0,0,0


In [7]:
def create_table2(df, label=None, pred=None, oracle=None, explain=False, judge=None):
	if oracle != None:
		df = df[(df['oracle_class'] == oracle)]
	if explain != None:
		df = df[(df['explain'] == explain)]
	if judge != None:
		df = df[(df['judge_name'] == judge)]

	if label == "A":
		df = df[(df['original_label'] == 'ResponseQuality.A_BETTER')]
	elif label == "B":
		df = df[(df['original_label'] == 'ResponseQuality.B_BETTER')]
	elif label == "TIE":
		df = df[(df['original_label'] == 'ResponseQuality.TIE')]

	if pred == 1:
		df = df[(df['pred_correct'] == 1)]
	elif pred == .5:
		df = df[(df['pred_correct'] == .5)]
	elif pred == 0:
		df = df[(df['pred_correct'] == 0)]

	init_data = [[0,0,0], [0,0,0], [0,0,0]]
	for index, row in df.iterrows():
		i = 0 if row['original_label'] == 'ResponseQuality.A_BETTER' else (1 if row['original_label'] == 'ResponseQuality.B_BETTER' else 2)
		j = int((row["pred_correct"])*2)
		init_data[i][j] += 1
	return pd.DataFrame(data=init_data,index=['A', 'B', 'TIE'], columns=['0', '.5', '1'])
	
	
create_table2(df,oracle='DiffOracle', explain=False, judge="Meta-Llama-3.1-70B-Instruct-q8_0")

,0,.5,1
A,14,0,41
B,11,0,29
TIE,0,0,0
